In [1]:
# Входные данные
inp_year = 2023
inp_month = 6

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import calendar as cal
from itertools import accumulate
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder \
      .master("local[2]") \
      .appName("SparkFirst")   \
      .config("spark.executor.memory", "32g") \
      .config("spark.driver.memory", "32g") \
      .config("spark.executor.cores", 4) \
      .config("spark.dynamicAllocation.enabled", "true") \
      .config("spark.dynamicAllocation.maxExecutors", 4) \
      .config("spark.shuffle.service.enabled", "true") \
      .getOrCreate()
df_dem = spark.read.option("header", True).option("inferSchema", "true").csv("data/2-demand.csv")
df_stk = spark.read.option("header", True).option("inferSchema", "true").csv("data/2-stock.csv")

# Без join'а можно обойтись, если мы уверены, что в обоих df построчное соответствие ключа (product, location)
df = df_dem\
  .join(df_stk, [ df_dem.product == df_stk.product, df_dem.location == df_stk.location ])\
  .select((df_dem.product).alias("prod"), (df_dem.location).alias("loc"), df_dem.demand, df_stk.stock)

del df_dem, df_stk

# Рассчитываем количество дней в технических неделях
d_1, d_tot = cal.monthrange(inp_year, inp_month)
week_len = [7 - d_1] + [7]*((d_1 + d_tot - 7)//7) + [(d_1 + d_tot) % 7]
if week_len[-1] == 0: del week_len[-1]
week_len_run = [0] + list(accumulate(week_len)) # нарастающий итог
print("Продолжительность технических недель:", week_len)

# Добавляем столбцы
def lbl(pf, n): return "wk" + str(n) + pf
new_cols_dict = dict(zip(
    [ lbl("-dem", n) for n in range(1, len(week_len) + 1) ] +              # заголовки - спрос
    ["op-stk"] + [ lbl("-stk", n) for n in range(1, len(week_len) + 1) ],  # заголовки - запас
    [ df['demand'] * wl for wl in week_len ] +                             # значения - спрос
    [ df['stock'] - df['demand'] * wlr for wlr in week_len_run ]           # значения - запас
))

df = df.withColumns(new_cols_dict).drop("demand", "stock")
print("Таблица по техническим неделям:")
df.show()

Продолжительность технических недель: [4, 7, 7, 7, 5]
Таблица по техническим неделям:
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|prod|loc|wk1-dem|wk2-dem|wk3-dem|wk4-dem|wk5-dem|op-stk|wk1-stk|wk2-stk|wk3-stk|wk4-stk|wk5-stk|
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|   1|  1|    400|    700|    700|    700|    500|  1000|    600|   -100|   -800|  -1500|  -2000|
|   1|  2|    440|    770|    770|    770|    550|   400|    -40|   -810|  -1580|  -2350|  -2900|
|   2|  1|    480|    840|    840|    840|    600|   300|   -180|  -1020|  -1860|  -2700|  -3300|
|   2|  2|    360|    630|    630|    630|    450|   250|   -110|   -740|  -1370|  -2000|  -2450|
|   3|  1|    280|    490|    490|    490|    350|   700|    420|    -70|   -560|  -1050|  -1400|
|   3|  2|    320|    560|    560|    560|    400|   800|    480|    -80|   -640|  -1200|  -1600|
|   4|  1|    160|    280|    28

In [4]:
# Нужно убрать отрицательные значения
@udf(returnType = IntegerType())
def non_neg(x): return 0 if x < 0 else x
new_cols_dict = { lbl("-stk", n) : non_neg(lbl("-stk", n)) for n in range(1, len(week_len) + 1) }
df = df.withColumns(new_cols_dict)
df.show()

+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|prod|loc|wk1-dem|wk2-dem|wk3-dem|wk4-dem|wk5-dem|op-stk|wk1-stk|wk2-stk|wk3-stk|wk4-stk|wk5-stk|
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|   1|  1|    400|    700|    700|    700|    500|  1000|    600|      0|      0|      0|      0|
|   1|  2|    440|    770|    770|    770|    550|   400|      0|      0|      0|      0|      0|
|   2|  1|    480|    840|    840|    840|    600|   300|      0|      0|      0|      0|      0|
|   2|  2|    360|    630|    630|    630|    450|   250|      0|      0|      0|      0|      0|
|   3|  1|    280|    490|    490|    490|    350|   700|    420|      0|      0|      0|      0|
|   3|  2|    320|    560|    560|    560|    400|   800|    480|      0|      0|      0|      0|
|   4|  1|    160|    280|    280|    280|    200|  2500|   2340|   2060|   1780|   1500|   1300|
|   4|  2|    300|  